<a href="https://colab.research.google.com/github/shenzhun/machine-learning-prep/blob/master/tensorflow/day27_model_deploying_using_tensorflow_serving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import *

2.4.0


In [3]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers

n = 800

X = tf.random.uniform([n, 2], minval=-10, maxval=10)
w0 = tf.constant([[2.0], [-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n, 1], mean=0.0, stddev=2.0)

tf.keras.backend.clear_session()
inputs = layers.Input(shape=(2,), name="inputs")
outputs = layers.Dense(1, name="outputs")(inputs)
linear = models.Model(inputs=inputs, outputs=outputs)
linear.summary()

# training with fit method
linear.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
linear.fit(X, Y, batch_size=8, epochs=100)

tf.print("w = ", linear.layers[1].kernel)
tf.print("b = ", linear.layers[1].bias)

export_path = "../data/linear_model/"
version = "1"
linear.save(export_path+version, save_format="tf")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 2)]               0         
_________________________________________________________________
outputs (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 1s 799us/step - loss: 255.2355 - mae: 13.0679
Epoch 2/100
100/100 [==============================] - 0s 779us/step - loss: 250.7670 - mae: 13.0620
Epoch 3/100
100/100 [==============================] - 0s 785us/step - loss: 217.6545 - mae: 12.0560
Epoch 4/100
100/100 [==============================] - 0s 786us/step - loss: 199.6380 - mae: 11.5615
Epoch 5/100
100/100 [==============================] - 0s 842us/step - loss: 170.5205 - mae: 10.6403
Epoch 6/100
100/100 [===

In [4]:
!ls {export_path+version}

assets	saved_model.pb	variables


In [6]:
!saved_model_cli show --dir {export_path+str(version)} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: serving_default_inputs:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['outputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W1229 05:28:35.296317 139864148289408 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/resource_vari

In [9]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  13882      0 --:--:-- --:--:-- --:--:-- 13882
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://devel

In [10]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 17 not upgraded.
Need to get 223 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.4.0 [223 MB]
Fetched 223 MB in 3s (67.0 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 145487 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.4.0_all.deb ...
Unpacking tensorflow-model-server (2.4.0) ...
Setting up tensorflow-model-server (2.4.0) ...


In [12]:
!nohup tensorflow_model_server --rest_api_port=8501 --model_name=linear_model --model_base_path=/data/linear_model/ > server.log 2>&1

^C


In [13]:
!curl -d '{"instances": [1.0, 2.0, 5.0]}' \
    -X POST http://localhost:8501/v1/models/linear_model:predict

curl: (7) Failed to connect to localhost port 8501: Connection refused


In [14]:
import json, requests

data = json.dumps({"signature_name": "serving_default",
                   "instances": [[1.0, 2.0], [5.0, 7.0]]})
headers = {"content-type": "application/json"}
json_response = requests.post("http://localhost:8501/v1/models/linear_model:predict",
                              data=data,
                              headers=headers)
predications = json.loads(json_response.text)["predications"]
print(predications)

ConnectionError: ignored